### Artsy API 



In [1]:
import numpy as np
import pandas as pd
import re
from time import sleep
from flatten_dict import flatten
from artsypy.api import Artsy

In [4]:
#!pip install vcrpy

#### Get Token

In [9]:
#!curl -v -X POST "https://api.artsy.net/api/tokens/xapp_token?client_id=<CLIENT_ID>&client_secret=<CLIENT_SECRET>"

* Expire in 0 ms for 6 (transfer 0x7f99b9801000)
* Expire in 1 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 1 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 1 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 1 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 1 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 1 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 1 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for 1 (transfer 0x7f99b9801000)
* Expire in 0 ms for

In [5]:
client_id = '<CLIENT_ID>'
client_secret = '<CLIENT_SECRET>'


In [12]:
import pytest
import vcr
import os
from artsypy.api import Artsy


tape = vcr.VCR(
    cassette_library_dir='tests/cassettes',
    serializer='json',
    record_mode='once'
)


key = os.environ.get('ARTSY_KEY')
secret = os.environ.get('ARTSY_SECRET')


def authenticate():
    artsy = Artsy(str(client_id), str(client_secret))

    return artsy


artsy = authenticate()


def test_get_auth_token():
    assert isinstance(artsy.auth_token, str)
    assert isinstance(artsy.token_expiry, str)


@vcr.use_cassette('tests/cassettes/get_artists.yml', filter_query_parameters=['key', 'secret'])
def test_get_artists():
    artist = 'Andy Warhol'

    r_json = artsy.get_artists(artist)

    assert isinstance(r_json['_links'], dict)

    r_str = artsy.get_artists(artist, return_type='text')

    assert isinstance(r_str, str)
    assert r_str[1:32] == '"id":"4d8b92b34eb68a1b2c0003f4"'

    r_similar = artsy.get_artists(similar_to_artist_id='4d8b92b34eb68a1b2c0003f4')

    assert isinstance(r_similar['_embedded']['artists'], list)

    r_artwork = artsy.get_artists(artwork_id='516dfb9ab31e2b2270000c45')

    assert r_artwork['_embedded']['artists'][0]['name'] == 'William Michael Harnett'

    with pytest.raises(ValueError):
        artsy.get_artists(artist, return_type='sjsfgerf')

In [6]:
artsy_obj=Artsy(client_id, secret = client_secret)

In [21]:
artist_dict = Artsy.get_artists(artsy_obj, artist="irene-aronson") 

### Import MoMa csv's

In [215]:
artworks_df = pd.read_csv('../../Museum_data/museum-collection/artworks.csv')
artists_df = pd.read_csv('../../Museum_data/museum-collection/artists.csv')
#met = pd.read_csv('MetObjects.csv')
female_ = artists_df[(artists_df.Gender == "Female")]
female_name = female_.Name.to_list()
female_name = [el.lower().replace(' ', '-') for el in female_name]

In [5]:
#artist_dict = {}

### Initial API pull - get artist info

In [219]:
for artist in range(len(female_name)):  
    artist_dict[female_name[artist]] = Artsy.get_artists(artsy_obj, artist=female_name[artist]) 
    sleep(1)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Flatten JSON dict 

In [222]:
artist_dict2 = flatten(artist_dict)


### Change keys to include only general key, combine dicts

In [237]:
new_dict = {}

In [238]:

for keys, values in artist_dict2.items():
    
        if len(keys) == 2:
            new_dict.setdefault(keys[1],[]).append(values)
            #dictionary[new_key] = dictionary[old_key]
            #del keys
        if len(keys) == 4:
            new_dict.setdefault(keys[2],[]).append(values)
            
        

In [225]:
print(len(new_dict['similar_contemporary_artists']))
print(len(new_dict['name']))
print(len(new_dict['gender']))
print(len(new_dict['nationality']))

92
92
92
92


In [190]:
artist_dict3 = {}


In [239]:
for keys in new_dict:
    if len(new_dict[keys]) != 92:
        print(keys)
        

type
message
image_versions
thumbnail
image


In [240]:
del new_dict['image_versions']

del new_dict['message']
del new_dict['type']
del new_dict['image']

### Create Artist Dataframe

In [241]:
artist_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in new_dict.items() ]))

In [6]:
#artist_df.drop('p','d','message', 'image', axis=1)

#### Grab artist ID to pass through API to get similar artists

In [257]:
similar_ids = artist_df.id.to_list()
#artist_df.name[2]
for artist in range(len(similar_ids)): 
    print(similar_ids[artist])

4e972c2fffe11800010004c7
4e96f7705554c900010027db
53ee6c7463617216f37c0000
55d62194726169212c0003f2
4f552b2e3b5552417000000a
52ce738dcd530eec850000f3
56dce374275b2441d800352d
4f552b2e3b55524170000010
4f552b2e3b55524170000015
4db5b4437798a6169500026f
50e5c01919c67a78160004e6
505a0531649d250002000d71
589ca6568b3b817c606cbbae
52701f20275b2440840004b9
4f07637834e11700010002c1
53a0acac7261691ef1200000
5122b9605f25167872000184
4e026cf52bd64f00010055b8
53344535c9dc241f5f000037
4e8f6394f0caa80001003ffa
544a461f7261694583d90300
4dd29a9617a37c0001004fd0
51f2df948b3b81e9c2000693
5151cf2576db51ac300002ff
4f552b2e3b55524170000041
4f5f64c13b555230ac00009c
58c2fb899c18db1329018eb6
50c84c1e623327ed6c000c24
5a9b29eb7622dd484c05694c
50d0bcf3ab7e6f725c0002d4
506b333144661700020004f2
4e8f63db069bb90001003fb5
597b86d9139b2132667c459a
4dd196c617a37c000100056a
4f5f64c13b555230ac0000be
4fd640695ee95000010002af
50082375f937d70002000fed
5c0ab035aa4e7c7b0c8ed432
56aa6eed9c18db667b000315
5059cf27f280130002000758


#### Get similar artists via API

In [263]:
similar_artist_dict = {}

In [264]:
for artist in range(len(similar_ids)):  
    similar_artist_dict[similar_ids[artist]] = Artsy.get_artists(artsy_obj, similar_to_artist_id=similar_ids[artist]) 
    sleep(1)
#Artsy.get_artists(artsy_obj, artist="53ee6c7463617216f37c0000") 

In [267]:
artist_df[artist_df.id == "4e972c2fffe11800010004c7"]

,id,slug,created_at,updated_at,name,sortable_name,gender,biography,birthday,deathday,...,location,nationality,thumbnail,self,permalink,artworks,published_artworks,similar_artists,similar_contemporary_artists,genes
0,4e972c2fffe11800010004c7,ruth-asawa,2011-10-13T18:21:35+00:00,2020-02-07T16:04:21+00:00,Ruth Asawa,Asawa Ruth,female,,1926,2013,...,None,Japanese-American,https://d32dm0rphc51dk.cloudfront.net/4FEktOvG...,https://api.artsy.net/api/artists/4e972c2fffe1...,https://www.artsy.net/artist/ruth-asawa,https://api.artsy.net/api/artworks?artist_id=4...,https://api.artsy.net/api/artworks?artist_id=4...,https://api.artsy.net/api/artists?similar_to_a...,https://api.artsy.net/api/artists?similar_to_a...,https://api.artsy.net/api/genes?artist_id=4e97...


In [268]:
#'4f5f64c13b555230ac00009c'
pd.DataFrame.from_dict(similar_artist_dict)

,4e972c2fffe11800010004c7,4e96f7705554c900010027db,53ee6c7463617216f37c0000,55d62194726169212c0003f2,4f552b2e3b5552417000000a,52ce738dcd530eec850000f3,56dce374275b2441d800352d,4f552b2e3b55524170000010,4f552b2e3b55524170000015,4db5b4437798a6169500026f,...,4dab297e0286243ad1000419,58f81e6f8b3b813376a30998,4f0648328501fa10ca00008e,4f0648328501fa10ca0000b1,4f5f64c13b555230ac000148,5d24785a9fa10023b11c2b03,52fe969ba09a6709db000439,4f0648328501fa10ca0000de,506b335044661700020007db,4e9c4c9b7c10480001007f2c
_embedded,{'artists': [{'id': '4d9e2f11f9215a26b3000104'...,{'artists': [{'id': '4f552b2e3b55524170000054'...,{'artists': []},{'artists': []},{'artists': [{'id': '53c9aea27261690a88000800'...,{'artists': [{'id': '50cf4c2db67733e128000ab9'...,{'artists': []},{'artists': [{'id': '52a1a823275b24e8a600034d'...,{'artists': [{'id': '4e94bcf5a67cea0001003702'...,{'artists': [{'id': '506b34234466170002001a7d'...,...,{'artists': [{'id': '4dab29db0286243ad100044d'...,{'artists': []},{'artists': [{'id': '4f0648328501fa10ca000084'...,{'artists': [{'id': '507c4435a49a1100020007a9'...,{'artists': [{'id': '5113b092b86c84b7650000b9'...,{'artists': [{'id': '5c0d0a346e47b0209b4df6bb'...,{'artists': []},{'artists': [{'id': '525932ff9c18dbe56f001095'...,{'artists': []},{'artists': [{'id': '4e94bcf5a67cea0001003702'...


In [276]:
new_embedded = flatten(similar_artist_dict['4e972c2fffe11800010004c7'])

In [299]:
similar_artist_dict['4e972c2fffe11800010004c7']['_embedded']['artists'][0]['name']

'Ben Shahn'

In [327]:
similar_artist_dict#['4e972c2fffe11800010004c7']['_embedded']['artists'][0])

{'4e972c2fffe11800010004c7': {'_embedded': {'artists': [{'id': '4d9e2f11f9215a26b3000104',
     'slug': 'ben-shahn',
     'created_at': '2011-04-07T21:39:29+00:00',
     'updated_at': '2020-02-07T07:19:28+00:00',
     'name': 'Ben Shahn',
     'sortable_name': 'Shahn Ben',
     'gender': 'male',
     'biography': None,
     'birthday': '1898',
     'deathday': '1969',
     'hometown': 'Kovno, Lithuania',
     'location': None,
     'nationality': 'American',
     'image_versions': ['four_thirds', 'large', 'square', 'tall'],
     '_links': {'thumbnail': {'href': 'https://d32dm0rphc51dk.cloudfront.net/yBICi1URKO16nr6LZVJ52g/four_thirds.jpg'},
      'image': {'href': 'https://d32dm0rphc51dk.cloudfront.net/yBICi1URKO16nr6LZVJ52g/{image_version}.jpg',
       'templated': True},
      'self': {'href': 'https://api.artsy.net/api/artists/4d9e2f11f9215a26b3000104'},
      'permalink': {'href': 'https://www.artsy.net/artist/ben-shahn'},
      'artworks': {'href': 'https://api.artsy.net/api/artwo

In [326]:
similar_artist_dict['4e972c2fffe11800010004c7']['_embedded']['artists'][0]['name']
similar_artist_dict['4e972c2fffe11800010004c7']['_embedded']['artists'][0]['_links']['permalink']['href']
similar_artist_dict['4e972c2fffe11800010004c7']['_embedded']['artists'][0]['_links']['thumbnail']['href']

'https://d32dm0rphc51dk.cloudfront.net/yBICi1URKO16nr6LZVJ52g/four_thirds.jpg'

### Isolate only certain fields from very nested dicts

In [353]:
master_list = []


In [354]:
for key in similar_artist_dict:
    artists = similar_artist_dict[key]['_embedded']['artists']
    names = []
    artist_link = []
    thumbnail = []
    for sub_dict in artists:
        names.append(sub_dict['name'])
        artist_link.append(sub_dict['_links']['permalink']['href'])
        thumbnail.append(sub_dict['_links']['thumbnail']['href'])
        
    master_list.append([key, names, artist_link, thumbnail])
    
sim_df_ = pd.DataFrame( master_list, columns = ['id', 'similar_names','similar_links','thumbnails'])
sim_df_.head()    

,id,similar_names,similar_links,thumbnails
0,4e972c2fffe11800010004c7,"[Ben Shahn, Dorothea Rockburne, Robert Rausche...","[https://www.artsy.net/artist/ben-shahn, https...",[https://d32dm0rphc51dk.cloudfront.net/yBICi1U...
1,4e96f7705554c900010027db,"[Bill Brandt, Paul Strand, Arnold Eagle, Walke...","[https://www.artsy.net/artist/bill-brandt, htt...",[https://d32dm0rphc51dk.cloudfront.net/s6uPnej...
2,53ee6c7463617216f37c0000,[],[],[]
3,55d62194726169212c0003f2,[],[],[]
4,4f552b2e3b5552417000000a,"[Barbara Stauffacher Solomon, Ernst Igl, Ilya ...",[https://www.artsy.net/artist/barbara-stauffac...,[https://d32dm0rphc51dk.cloudfront.net/IJBB4db...


In [343]:
len(sim_df.similar_names[2])

0

### Merge Datasets on ID

In [365]:
#df[df.astype(str)['donation_orgs'] != '[]']
sim_df_ = sim_df_[sim_df_.astype(str)['similar_names'] != "[]"]

In [367]:
all_artists_df = pd.merge(artist_df, sim_df_)
all_artists_df.drop("published_artworks", "similar_artists", "similar_contemporary_artists", "sortable_name","biography", "genes", axis=0)
#sim_df_
#artist_df

TypeError: drop() got multiple values for argument 'axis'

In [373]:
#all_artists_df.drop("published_artworks", "similar_artists", "similar_contemporary_artists", "sortable_name","biography", "genes", axis=1)


all_artists_df = all_artists_df.loc[:,["id", "name", "gender", "nationality", "permalink", "thumbnail", "similar_names", "similar_links", "thumbnails"]]


In [376]:
all_artists_df.to_csv("artsy_api.csv")

In [351]:
sim_df = sim_df.dropna()

In [352]:
sim_df

,id,similar_names,similar_links,thumbnails
0,4e972c2fffe11800010004c7,"[Ben Shahn, Dorothea Rockburne, Robert Rausche...","[https://www.artsy.net/artist/ben-shahn, https...",[https://d32dm0rphc51dk.cloudfront.net/yBICi1U...
1,4e96f7705554c900010027db,"[Bill Brandt, Paul Strand, Arnold Eagle, Walke...","[https://www.artsy.net/artist/bill-brandt, htt...",[https://d32dm0rphc51dk.cloudfront.net/s6uPnej...
2,53ee6c7463617216f37c0000,[],[],[]
3,55d62194726169212c0003f2,[],[],[]
4,4f552b2e3b5552417000000a,"[Barbara Stauffacher Solomon, Ernst Igl, Ilya ...",[https://www.artsy.net/artist/barbara-stauffac...,[https://d32dm0rphc51dk.cloudfront.net/IJBB4db...
5,52ce738dcd530eec850000f3,"[Shirley Clarke, Louise Fishman, Christina Ram...","[https://www.artsy.net/artist/shirley-clarke, ...",[https://d32dm0rphc51dk.cloudfront.net/A30iIu5...
6,56dce374275b2441d800352d,[],[],[]
7,4f552b2e3b55524170000010,"[Maria Thereza Alves, Claudio Edinger, Dias & ...",[https://www.artsy.net/artist/maria-thereza-al...,[https://d32dm0rphc51dk.cloudfront.net/LUM5iRh...
8,4f552b2e3b55524170000015,"[Nancy Spero, Andrea Fraser, Sanja Iveković, K...","[https://www.artsy.net/artist/nancy-spero, htt...",[https://d32dm0rphc51dk.cloudfront.net/KLWT8Bt...
9,4db5b4437798a6169500026f,"[Rosalind Solomon, Al Vandenberg, Milton Rogov...",[https://www.artsy.net/artist/rosalind-solomon...,[https://d32dm0rphc51dk.cloudfront.net/PrkYixJ...
